In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

from time import perf_counter 

# Modeling

In [2]:
# loading training set
train_set = image_dataset_from_directory(
    './data/images/train/',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 100,
    image_size = (416,416)
)

Found 6051 files belonging to 3 classes.


In [3]:
# load test set
test_set = image_dataset_from_directory(
    './data/images/test/',
    label_mode = 'categorical',
    color_mode = 'rgb',
    batch_size = 100,
    image_size = (416,416)
)

Found 1986 files belonging to 3 classes.


In [4]:
train_set.class_names

['both', 'hard_hat', 'not_hh']

In [5]:
test_set.class_names

['both', 'hard_hat', 'not_hh']

## NN

In [6]:
cnn = Sequential(layers.BatchNormalization())

In [7]:
cnn.add(Conv2D(filters = 16,
              kernel_size = (3,3),
              activation = 'relu',
              input_shape = (416,416,1)))

In [8]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [9]:
cnn.add(Conv2D(filters = 32,
                     kernel_size = (3,3),
                     activation = 'relu'))

In [10]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [11]:
cnn.add(Conv2D(filters = 64,
                     kernel_size = (3,3),
                     activation = 'relu'))

In [12]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [13]:
cnn.add(Conv2D(filters = 128,
                     kernel_size = (3,3),
                     activation = 'relu'))

In [14]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [15]:
cnn.add(Conv2D(filters = 256,
                     kernel_size = (3,3),
                     activation = 'relu'))

In [16]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [17]:
cnn.add(Conv2D(filters = 512,
              kernel_size = (3,3),
              activation = 'relu',
              input_shape = (416,416,1)))

In [18]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [19]:
cnn.add(Conv2D(filters = 1024,
                     kernel_size = (3,3),
                     activation = 'relu'))

In [20]:
cnn.add(MaxPooling2D(pool_size = (2,2)))

In [21]:
cnn.add(Flatten())

In [22]:
cnn.add(Dense(2048, activation = 'relu'))

In [23]:
cnn.add(Dense(3,activation = 'softmax'))

In [24]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy','Recall','Precision'])

In [25]:
early_stop = EarlyStopping(monitor = 'accuracy',min_delta = 0, 
                           patience = 5,verbose = 1,mode = 'auto')

In [26]:
cnn.fit(train_set, validation_data = test_set,epochs = 5,workers = 12, callbacks = early_stop)

Epoch 1/5
61/61 [==============================] - 668s 11s/step - loss: 0.9474 - accuracy: 0.7019 - recall: 0.4866 - precision: 0.7114 - val_loss: 0.8067 - val_accuracy: 0.7185 - val_recall: 0.7185 - val_precision: 0.7185
Epoch 2/5
61/61 [==============================] - 639s 10s/step - loss: 0.7535 - accuracy: 0.7110 - recall: 0.7029 - precision: 0.7154 - val_loss: 0.7235 - val_accuracy: 0.7185 - val_recall: 0.7160 - val_precision: 0.7266
Epoch 3/5
61/61 [==============================] - 645s 11s/step - loss: 0.6867 - accuracy: 0.7226 - recall: 0.6788 - precision: 0.7608 - val_loss: 0.6440 - val_accuracy: 0.7603 - val_recall: 0.7341 - val_precision: 0.7772
Epoch 4/5
61/61 [==============================] - 640s 10s/step - loss: 0.6286 - accuracy: 0.7597 - recall: 0.7321 - precision: 0.7833 - val_loss: 0.6299 - val_accuracy: 0.7734 - val_recall: 0.7563 - val_precision: 0.7803
Epoch 5/5
61/61 [==============================] - 637s 10s/step - loss: 0.5723 - accuracy: 0.7857 - recall:

In [28]:
cnn.save('./streamlit_app/models/site_safety')

INFO:tensorflow:Assets written to: original/assets


In [29]:
preds = cnn.predict(test_set)

In [30]:
classes = np.argmax(preds, axis = 1)
print(classes)

[1 1 1 ... 1 1 2]


In [31]:
preds = pd.DataFrame(classes)

In [32]:
preds[0].value_counts()

1    1434
2     529
0      23
Name: 0, dtype: int64

In [33]:
test_df = pd.read_csv('./data/_test_classes.csv',index_col = 0)
test_df

,filename,not_hard_hat,hard_hat,target
0,005298_jpg.rf.7d1070e4a462d4c636a68e64f2013761...,0,1,0
1,005299_jpg.rf.a79cacc62cdd99ad871cf1786ad865ff...,0,1,0
2,005300_jpg.rf.419d88422f77db5d30bbb0e769c7c600...,1,1,2
3,005301_jpg.rf.c9590992f2ae3693821268735bc7191b...,1,0,1
4,005302_jpg.rf.3f9ff0dcbfa01ac541e96d7aa79685ff...,0,1,0
...,...,...,...,...
1761,007059_jpg.rf.abefc5190c7d3e869d026750d345ac3a...,0,1,0
1762,007060_jpg.rf.3a70f64aeb8fa4376eb4c039eff278db...,0,1,0
1763,007061_jpg.rf.1668007dcbe41891c62bcd48d7b03d4c...,0,1,0
1764,007062_jpg.rf.7e538c4f7f163acbec263ad8f0ac98da...,1,1,2


In [34]:
test_df['preds'] = preds[0]
test_df

,filename,not_hard_hat,hard_hat,target,preds
0,005298_jpg.rf.7d1070e4a462d4c636a68e64f2013761...,0,1,0,1
1,005299_jpg.rf.a79cacc62cdd99ad871cf1786ad865ff...,0,1,0,1
2,005300_jpg.rf.419d88422f77db5d30bbb0e769c7c600...,1,1,2,1
3,005301_jpg.rf.c9590992f2ae3693821268735bc7191b...,1,0,1,1
4,005302_jpg.rf.3f9ff0dcbfa01ac541e96d7aa79685ff...,0,1,0,1
...,...,...,...,...,...
1761,007059_jpg.rf.abefc5190c7d3e869d026750d345ac3a...,0,1,0,1
1762,007060_jpg.rf.3a70f64aeb8fa4376eb4c039eff278db...,0,1,0,2
1763,007061_jpg.rf.1668007dcbe41891c62bcd48d7b03d4c...,0,1,0,2
1764,007062_jpg.rf.7e538c4f7f163acbec263ad8f0ac98da...,1,1,2,1


In [35]:
tp = []

for x in range(len(test_df)):
    if test_df['preds'][x] != test_df['hard_hat'][x]:
        tp.append(x)

In [36]:
fn = []

trp = []

for x in tp:
    if test_df['preds'][x] == 1:
        fn.append(x)
    else:
        trp.append(x)

In [37]:
len(fn)

117

In [38]:
len(trp)

486

In [39]:
true_positive = []

for x in trp:
    if test_df['hard_hat'][x] == 1:
        pass
    else:
        true_positive.append(x)

In [40]:
len(true_positive)

43

In [41]:
print(true_positive)

[81, 192, 218, 243, 275, 281, 283, 295, 296, 308, 336, 349, 412, 481, 496, 520, 559, 614, 756, 758, 763, 840, 845, 850, 883, 950, 981, 1043, 1053, 1182, 1209, 1302, 1321, 1322, 1470, 1491, 1523, 1572, 1575, 1627, 1675, 1738, 1758]


In [42]:
for x in true_positive:
    print(test_df['filename'][x])

005379_jpg.rf.2ed9c87be4989d21a12c61b0a05e6398.jpg
005490_jpg.rf.9e481bdf97e64a0ae874db1eb8468553.jpg
005516_jpg.rf.280220e4da23859c5f995c76c2c7c7b7.jpg
005541_jpg.rf.360b6e3dd60609078153838d1cf8ed71.jpg
005573_jpg.rf.b0b1a44884952dbb14d3c753ff08665a.jpg
005579_jpg.rf.88d82901397aa90f11be6f1a5082a599.jpg
005581_jpg.rf.c18f45b7e989fb90cc5bd61819785148.jpg
005593_jpg.rf.875f0a552fb0734863f088a22cf3d5e1.jpg
005594_jpg.rf.9e634fb9ed2e9299b23dce2fe9926556.jpg
005606_jpg.rf.c683a4fe27ba31460a818e2f3d6097b6.jpg
005634_jpg.rf.883500b6e17d64ad0e84bb285d74ab17.jpg
005647_jpg.rf.7808ae0808aec7df7a33a1c4a9c09494.jpg
005710_jpg.rf.bdb3363adbfc7f38dcda8f60fa72bb88.jpg
005779_jpg.rf.f620055fdf80f168f17ee7d9c16e4096.jpg
005794_jpg.rf.eae06a1f3608ace909c65fef0180a658.jpg
005818_jpg.rf.23e0af08bfb3db3501759d4aa4364f7e.jpg
005857_jpg.rf.c6175dfbb046f790ac0a2612dc9a3478.jpg
005912_jpg.rf.0232549b729fa6a445e2cc9dfe47e92e.jpg
006054_jpg.rf.236141ff51d09b0b696ee5afcbd6d0dd.jpg
006056_jpg.rf.01114a2b771006d0f